In [1]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import cv2

Using TensorFlow backend.


In [2]:
%%time
raw_data_path = "./SCUT-FBP5500_v2/train_test_files/All_labels.txt"
raw_data = np.loadtxt(raw_data_path, delimiter=" ", unpack=True, dtype=str)
X_names = raw_data[0]
Y = raw_data[1]
Y = Y.astype(float)

X = []
#X = np.empty((5500, 350, 350, 3))
#print(X)
i = 0
for i,img_name in enumerate(X_names):
    img = cv2.imread("./SCUT-FBP5500_v2/Images/" + img_name)
    #X[i] = img
    X.append(img)
    i += 1
    if i == 100000:
        break

X = np.array(X)
print(X.shape)

(5500, 350, 350, 3)
CPU times: user 19.7 s, sys: 4.6 s, total: 24.3 s
Wall time: 31.4 s


In [3]:
%%time
# Splitting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

CPU times: user 1.51 s, sys: 2.88 s, total: 4.38 s
Wall time: 10.3 s


In [4]:
# Model initiation : resnet50 + denser layer with one cell (output) for regression
resnet = ResNet50(include_top=False, pooling="avg")
model = Sequential()
model.add(resnet)
model.add(Dense(1))

# We train only the last connections for now
model.layers[0].trainable = False

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 23,589,761
Trainable params: 2,049
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [5]:
%%time
# On précise la formule de l'erreur et la technique d'optimisation
model.compile(loss='mean_squared_error', optimizer=Adam())
# Entrainement
model.fit(batch_size=32, x=X_train, y=y_train, epochs=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
4400/4400 [==============================] - 6309s 1s/step - loss: 0.8992
CPU times: user 5h 20min 16s, sys: 18min 11s, total: 5h 38min 28s
Wall time: 1h 45min 9s


In [ ]:
import h5py
model.save('./model')

In [ ]:
scores = model.evaluate(X, Y, verbose=0)